In [ ]:
import pandas as pd
import numpy as np
import string
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, Dense, Flatten, Dropout, TimeDistributed, SimpleRNN, GlobalMaxPooling1D
from tensorflow.keras.metrics import AUC, Accuracy
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from functools import partial
import nltk
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, Embedding, LSTM , Dense, Flatten, Dropout , GRU, TimeDistributed, Conv1D, MaxPool1D
from keras.callbacks import TensorBoard, EarlyStopping
from tensorflow.keras import backend as K
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score,roc_curve, classification_report
from keras.optimizers import Adam

stop_words = stopwords.words('english')

In [ ]:
df = pd.read_csv("/content/dataset_train.csv")

In [ ]:
dataset["label"] = dataset["label"].astype(str)
dataset["premise"] = dataset["premise"].astype(str)
dataset["hypothesis"] = dataset["hypothesis"].astype(str)

In [ ]:
train_df, val_df = train_test_split(dataset, test_size=0.2, random_state = 42)
train_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True, inplace=True)
print("Train data size: ", train_df.size)
print("Validation data size: ", val_df.size)

In [ ]:
sentence1 = dataset["hypothesis"]
sentence2 = dataset["premise"]
label = dataset["label"]
labels = to_categorical(label.values, num_classes = 3)

In [ ]:
def CleanFeatures(sentences):
      sentences = sentences.apply(lambda sequence:
                                            [ltrs.lower() for ltrs in sequence if ltrs not in string.punctuation])
      sentences = sentences.apply(lambda wrd: ''.join(wrd))
      sentences = sentences.apply(lambda sequence:
                                            [word for word in sequence.split() if word not in stop_words])
      sentences = sentences.apply(lambda wrd: ' '.join(wrd))
      return sentences

sentence1 = CleanFeatures(sentence1)
sentence2 = CleanFeatures(sentence2)

In [ ]:
all_sentences = np.asarray([sentence1, sentence2])
all_sentences.shape
all_sentences = all_sentences.reshape(-1,1)
all_sentences = all_sentences.reshape(all_sentences.shape[0])

In [ ]:
tokenizer = Tokenizer(num_words=6000)
tokenizer.fit_on_texts(all_sentences)
sentence1_seq = tokenizer.texts_to_sequences(sentence1)
sentence1 = pad_sequences(sentence1_seq)

sentence2_seq = tokenizer.texts_to_sequences(sentence2)
sentence2 = pad_sequences(sentence2_seq, maxlen= sentence1.shape[1])

In [ ]:
vocabulary = len(tokenizer.word_index)

In [ ]:
all_sentences = np.asarray([sentence1, sentence2])
all_sentences.shape

In [ ]:
all_sentences = all_sentences.reshape(all_sentences.shape[1], 2, all_sentences.shape[2])
all_sentences.shape

In [ ]:
def glove_word_embedding(file_name, vocabulary):
      embeddings_index = {}
      file_ = open(file_name)
      for line in file_:
          arr = line.split()
          single_word = arr[0]
          w = np.asarray(arr[1:],dtype='float32')
          embeddings_index[single_word] = w
      file_.close()
      max_words = vocabulary + 1
      word_index = tokenizer.word_index
      embedding_matrix = np.zeros((max_words,50)).astype(object)
      for word , i in word_index.items():
              embedding_vector = embeddings_index.get(word)
              if embedding_vector is not None:
                  embedding_matrix[i] = embedding_vector 
      return embedding_matrix

In [ ]:
def f1_macro(y_true, y_pred):
    
    y_pred_labels = tf.argmax(y_pred, axis=1, output_type=tf.int32)
    f1 = tf.py_function(
        partial(f1_score, average='macro'),
        (tf.cast(y_true, tf.int64), y_pred_labels),
        tf.float64)

    return f1

In [ ]:
x = Input(shape=(all_sentences.shape[1], all_sentences.shape[2],))
p = TimeDistributed(Embedding(vocabulary + 1, 50))(x)
p = TimeDistributed(LSTM(128, return_sequences=True))(p)
p = TimeDistributed(LSTM(64, return_sequences=True))(p)
x1 = TimeDistributed(tf.keras.layers.GlobalMaxPooling1D())(p)
x2 = TimeDistributed(tf.keras.layers.GlobalAveragePooling1D())(p)
concat = tf.keras.layers.concatenate([x1, x2])
p = tf.keras.layers.Dropout(0.2)(concat)
p = Flatten()(p)
p = Dense(3, activation="softmax")(p)
m = tf.keras.models.Model(inputs = x, outputs = p)
m.layers[1].set_weights([glove_word_embedding("/content/glove.6B.50d.txt", vocabulary)])
m.layers[1].trainable = False

In [ ]:
def custom_f1(y_true, y_pred):
    def recall_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))

        recall = TP / (Positives+K.epsilon())
        return recall


    def precision_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))

        precision = TP / (Pred_Positives+K.epsilon())
        return precision

    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)

    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
m.compile(loss="categorical_crossentropy",
              optimizer=Adam(lr=0.001),
              metrics=[custom_f1])

In [ ]:
earlyStopping = EarlyStopping(monitor = 'loss', patience = 30, mode = 'min', restore_best_weights = True)

In [ ]:
training = m.fit(all_sentences,labels,
                         epochs= 450,
                         batch_size = 1024,
                         use_multiprocessing=True,
                        callbacks= [earlyStopping],verbose=0)

# Validation

In [ ]:
y_true = val_df["label"].astype(int)

In [ ]:
def pre_processing_sentence(sentence):

      texts = []
      pred_input = sentence
      pred_input = [letters.lower() for letters in pred_input if letters not in string.punctuation]
      pred_input = ''.join(pred_input)

      texts.append(pred_input)
      pred_input = tokenizer.texts_to_sequences(texts)
      pred_input = np.array(pred_input).reshape(-1)
      pred_input = pad_sequences([pred_input],62)

      return pred_input

def process_dataframe(df_test):

    preprocessed_data = []

    for index, row in df_test.iterrows():

        premise = pre_processing_sentence(row['premise'])
        hypothesis = pre_processing_sentence(row['hypothesis'])
        preprocessed_data.append([premise, hypothesis])

    preprocessed_data = np.array(preprocessed_data)
    preprocessed_data = preprocessed_data.reshape(-1, 2, 62)
    return preprocessed_data

In [ ]:
processed_data = process_dataframe(val_df)

y_predict = m.predict(processed_data)

In [ ]:
pred_flat = np.argmax(y_predict, axis=1).flatten()
pred_flat = pred_flat - 1

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score,roc_curve, classification_report

f1 = f1_score(y_true, pred_flat, average='macro')
accuracy= accuracy_score(y_true, pred_flat)

print("F1 score on validation set:", f1)
print("Accuracy score on validatin set:", accuracy)
print('Classification Report for LSTM :\n', classification_report(y_true, pred_flat))